# Full fine-tuning of a BERT SLM for Classification

![](https://i.imgur.com/7SXKckD.png)

Transfer Learning is the power of leveraging already trained models and tune \ adapt them to our own downstream tasks. 

Here we start by understanding how to fine-tune a simple BERT Small Language Model (SLM) step by step for a simple yet essential task in NLP - Text Classification for Sentiment Analysis 

# Sentiment Analysis

When it comes to text data, sentiment analysis is one of the most widely performed analysis on it. Sentiment Analysis has been through tremendous improvements from the days of classic methods to recent times where in the state of the art models utilize deep learning to improve the performance.

# Fine-tuning a model on a text classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a text classification task of Sentiment Analysis

![](https://i.imgur.com/Pq7f3Fd.png)

___[Created By: Dipanjan (DJ)](https://www.linkedin.com/in/dipanjans/)___

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

You will be leveraging 🤗 Transformers and 🤗 Datasets as well as other dependencies

## Get Dataset

In [ ]:
import pandas as pd

dataset = pd.read_csv(r'https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2011%20-%20Sentiment%20Analysis%20-%20Unsupervised%20Learning/movie_reviews.csv.bz2',
                      compression='bz2',
                      nrows=20000)
dataset.info()

In [ ]:
dataset['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in dataset['sentiment']]

In [ ]:
dataset.head()

## Create Training Datasets

This is a labeled dataset of IMDB movie reviews and their corresponding sentiment (1 or 0) which basically means (positive or negative).

Idea is to make BERT learn to predict the sentiment given the review.

Let's create some datasets first!

In [ ]:
train_df = dataset.iloc[:8000]
val_df = dataset.iloc[8000:10000]
test_df = dataset.iloc[10000:]

In [ ]:
train_df.sentiment.value_counts()

In [ ]:
val_df.sentiment.value_counts()

In [ ]:
test_df.sentiment.value_counts()

In [ ]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)
test_df.to_csv('test.csv', index=False)

## Load Dataset

Here we convert the CSV dataset files above into a huggingface dataset which is easier to use when training transformer models in huggingface

In [ ]:
from datasets import load_dataset, load_metric

data_files = {"train": "train.csv",
              "validation": "val.csv",
              "test": "test.csv"}
imdb_data = load_dataset("csv", data_files=data_files)

In [ ]:
imdb_data

In [ ]:
imdb_data.keys()

In [ ]:
# Optional you can push the dataset splits to HF hub
# and download in the future directly
# no need to do this for now as I have already done this
# we will use this from the next time for some of the tutorials in the next module
#imdb_data.push_to_hub("dipanjanS/imdb_sentiment_finetune_dataset20k")

In [ ]:
imdb_data['train'][:2]

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [ ]:
imdb_data

To access an actual element, you need to select a split first, then give an index:

In [ ]:
imdb_data["train"][0]

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the function `evaluate.load`.  

In [ ]:
import evaluate

metric1 = evaluate.load("precision")
metric2 = evaluate.load("recall")
metric3 = evaluate.load("f1")
metric4 = evaluate.load("accuracy")

def evaluate_performance(predictions, references):
    precision = metric1.compute(predictions=predictions, references=references, average="macro")["precision"]
    recall = metric2.compute(predictions=predictions, references=references, average="macro")["recall"]
    f1 = metric3.compute(predictions=predictions, references=references, average="macro")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=references)["accuracy"]
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}


You can call its `compute` method with your predictions and labels, which need to be list of decoded strings:

For classification most common metrics include accuracy and f1-score.


In [ ]:
predictions = [1,0,1,1,0]
references = [1,1,0,1,0]
scores = evaluate_performance(
    predictions=predictions, references=references
)
scores


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

This notebook is built to run on any of the tasks in the list above, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a classification head.

Here we picked the [`bert-base-uncased`](https://huggingface.co/bert-base-uncased) checkpoint.

![](https://i.imgur.com/GmFRcP3.png)

BERT can be used for a variety of tasks and we will fine-tune it for classification (sentiment).

Here we will use a smaller version of the BERT model called DistilBERT to train faster.

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer("Hello, this is a sentence!")

We can then write the function that will preprocess our samples. We just feed them to the `tokenizer` with the argument `truncation=True`.

This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

The padding will be dealt with later on (in a data collator) so we pad examples to the longest length in the batch and not the whole dataset.

In [ ]:
def preprocess_function(examples):
    # max length is 512 as that is the context window limit of BERT models
    # It can process documents of upto 512 tokens each input
    model_inputs = tokenizer(examples['review'], max_length=512, truncation=True)
    model_inputs["label"] = examples["sentiment"]
    return model_inputs

This function works with one or several documents. In the case of several documents, the tokenizer will return a list of lists for each key:

In [ ]:
preprocess_function(imdb_data["train"][:2])

To apply this function on all the sentences in our dataset, we just use the `map` method of our `dataset` object we created earlier.

This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [ ]:
tokenized_datasets = imdb_data.map(preprocess_function, batched=True)

In [ ]:
# remove unnecessary columns
tokenized_datasets = tokenized_datasets.remove_columns('review')
tokenized_datasets = tokenized_datasets.remove_columns('sentiment')

## Fine-tuning the Transformer Model

Now that our data is ready, we can download the pretrained model and fine-tune it.

Since our task is about sentence classification, we use the `AutoModelForSequenceClassification` class.

Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

The only thing we have to specify is the number of labels for our problem which should be 2

In [ ]:
# we put in a mapping so the model knows which prediction label ID is which text label (human friendly)
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = <YOUR CODE HERE>

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers).

This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights.

So the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do to make the model learn how to predict the two classes

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
print_trainable_parameters(model)

The above piece of code shows us that DistilBERT has 66 Million trainable parameters and we are training all of them here which is basically what full fine-tuning means.

To instantiate a `Trainer`, we will need to define two more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
# if batch size is 64
# if total documents are 8000
# total number of steps (batches of data) to complete 1 full epoch is?
<YOUR CODE HERE>

In [ ]:
# total steps to run two epochs are?
<YOUR CODE HERE>

In [ ]:
from transformers import TrainingArguments

batch_size = 64 
metric_name = "f1"

# Set up the training arguments
args = TrainingArguments(
    output_dir="distilbert-fullfinetune-runs",  # Directory where the model checkpoints and outputs will be saved.
    eval_strategy="steps",                      # Perform evaluation at regular intervals during training.
    save_strategy="steps",                      # Save the model checkpoint at regular intervals.
    learning_rate=1e-5,                         # Initial learning rate for the optimizer.
    logging_steps=20,                           # Log training metrics every 20 steps.
    eval_steps=20,                              # Perform evaluation every 20 steps.
    save_steps=50,                              # Save the model checkpoint every 50 steps.
    per_device_train_batch_size=batch_size,     # Batch size per GPU/TPU core/CPU during training.
    per_device_eval_batch_size=batch_size,      # Batch size per GPU/TPU core/CPU during evaluation.
    max_steps=250,                              # Stop training after 250 total steps.
    weight_decay=0.01,                          # Apply weight decay to reduce overfitting.
    metric_for_best_model=metric_name,          # Metric to use for selecting the best model during evaluation.
    push_to_hub=False                           # Do not push the model to the Hugging Face Hub after training.
)

We use DataCollatorWithPadding to create a batch of examples. It will also dynamically pad your text to the length of the longest element in its batch, so they are a uniform length.

While it is possible to pad your text in the tokenizer function by setting `padding=True`, dynamic padding is more efficient.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions.

We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits.

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return evaluate_performance(predictions=predictions, references=labels)

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

Run and wait for around 5-6 mins on a 48GB GPU

In [ ]:
trainer.train()

## Save and Load Fine-tuned BERT Model

In [ ]:
save_path = 'fullfinetune-distilbert-classification'
<YOUR CODE HERE>

In [ ]:
# remove model checkpoints
!rm -rf distilbert-fullfinetune-runs

In [ ]:
loaded_model = <YOUR CODE HERE>
loaded_tokenizer = <YOUR CODE HERE>

# Using your fine-tuned model for Classification

Once you’ve fine-tuned the model you can use it with a pipeline object, for inference as follows:

In [ ]:
from transformers import pipeline

In [ ]:
# Here you can load your locally trained \ saved model
clf = pipeline(task='text-classification', 
               model=loaded_model, 
               tokenizer=loaded_tokenizer, 
               device='cuda')

In [ ]:
document = "The movie was not good at all"

In [ ]:
clf(document)

In [ ]:
document = "The movie was amazing"

In [ ]:
clf(document)

## Fine-tuned Transformer performance on Test Data

We can feed our test set (which the model has not seen) to our pipeline to get a feel for the quality of the model predictions.

In [ ]:
imdb_data['test'][:2]

Inference on the full test data takes roughly 1-2 mins

In [ ]:
%%time

predictions = clf(imdb_data['test']['review'],
                  batch_size=512, 
                  max_length=512, 
                  truncation=True)
predictions = [pred['label'] for pred in predictions]

predictions = [0 if item == 'NEGATIVE' else 1 for item in predictions]
labels = imdb_data['test']['sentiment']

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(labels, predictions))
pd.DataFrame(confusion_matrix(labels, predictions))